In [1]:
from aux_train import *
from aux_plot import *
from IPython.display import clear_output

In [2]:
import wandb
from wandb.keras import WandbCallback

In [3]:
# config = wandb.config
# config.equation='tranp_equation'
# config.WENO='WENO-Z-DS'

# config.poly_grade=7
# config.seno_ampli=3 
# config.gauss_var=3
# config.seizures=3

sweep_config = {
    'method': 'bayes',
    'metric': {
      'name': 'validation_error',
      'goal': 'minimize'   
    },
    'parameters': {
        'poly_grade': {
            'values': list(range(8))
        },
        'seno_ampli': {
            'values': list(range(4))
        },
        'gauss_var': {
            'values': list(range(4))

        },
        'seizures': {
            'values': list(range(4))
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project='WENO-DS', entity='silvaneojunior')
#sweep_id = 'silvaneojunior/LSTM Poemas/iuazw8n3'

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: hfp7oaid
Sweep URL: https://wandb.ai/silvaneojunior/WENO-DS/sweeps/hfp7oaid


In [4]:
default_equation='transp_equation'
default_WENO='WENO-Z-DS'

default_poly_grade=7
default_seno_ampli=3 
default_gauss_var=3
default_seizures=3

In [5]:
def f_test(x):
    z = tf.constant(-0.7, dtype=float_pres)
    δ = tf.constant(0.005, dtype=float_pres)
    β = tf.math.log(tf.constant(2.0, dtype=float_pres))/(36.0*(δ**2.0))
    a = tf.constant(0.5, dtype=float_pres)
    α = tf.constant(10, dtype=float_pres)
    
    def G(x, β, z):
        return tf.math.exp(-β*(x-z)**2)
    
    def F(x, α, a):
        return tf.math.sqrt(tf.math.maximum(1-(α**2)*((x-a)**2),0))
    
    f1 = (G(x, β, z-δ) + 4*G(x, β, z) + G(x, β, z+δ))/6 # x in [-0.8, -0.6]
    f2 = 1                                              # x in [-0.4, -0.2]
    f3 = 1-tf.math.abs(10*(x-0.1))                      # x in [ 0.0,  0.2]
    f4 = (F(x, α, a-δ) + 4*F(x, α, a) + F(x, α, a+δ))/6 # x in [ 0.4,  0.6]
    f5 = 0                                              # otherwise
    
    condition = tf.math.logical_and(
        tf.math.greater_equal(x, -0.8),
        tf.math.less_equal(x, -0.6)
    )
    f1 = f1*tf.cast(tf.where(condition, 1.0, 0.0), dtype=float_pres)
    
    condition = tf.math.logical_and(
        tf.math.greater_equal(x, -0.4),
        tf.math.less_equal(x, -0.2)
    )
    f2 = f2*tf.cast(tf.where(condition, 1.0, 0.0), dtype=float_pres)
    
    condition = tf.math.logical_and(
        tf.math.greater_equal(x, 0.0),
        tf.math.less_equal(x, 0.2)
    )
    f3 = f3*tf.cast(tf.where(condition, 1.0, 0.0), dtype=float_pres)
    
    condition = tf.math.logical_and(
        tf.math.greater_equal(x, 0.4),
        tf.math.less_equal(x, 0.6)
    )
    f4 = f4*tf.cast(tf.where(condition, 1.0, 0.0), dtype=float_pres)
    
    f = f1 + f2 + f3 + f4 + f5
    
    return f

x,fx=create_f_points(f_test=f_test,Δx=0.01)

In [6]:
data_temporal,data_spatial,data_base,Δx,Δt,CFL,fronteira,equation=load_dataset(f'datasets/equation={default_equation} polys grau={default_poly_grade} seno_ampli={default_seno_ampli} gauss_var={default_gauss_var} seizure={default_seizures}')

indice = np.arange(data_temporal.shape[0])
np.random.shuffle(indice)
data_x = data_temporal.astype('float64')[indice]
data_y = data_temporal.astype('float64')[indice]

train_x = data_x[:,0:-1]
train_y = data_y[:,1:]

'Δx': 0.015625
'Δt': 0.004
'CFL': 0.5
'fronteira': <function FronteiraPeriodica at 0x0000015D86B7F550>
'equation': <class 'aux_equation.transp_equation'>
'seed': None



In [7]:
Sim_layer = WENO_temporal_layer(equation,WENO_Z,Δx=Δx,Δt=Δt,fronteira=fronteira,regul_weight=0,ativ_func=tf.nn.sigmoid,p=2,conv_size=5)

input_x   = keras.layers.Input(train_x.shape[1:], dtype='float64')
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, clipnorm=1.0)

output_x=keras.layers.TimeDistributed(Sim_layer)(input_x)

# Criando a rede neural
Network = keras.Model(input_x, output_x)
Network.compile(loss='MSE', optimizer=optimizer, metrics=['mean_absolute_error'])

def reset_weights(model):
    for layer in model.layers:
        if isinstance(layer, tf.keras.Model): #if you're using a model as a layer
            reset_weights(layer) #apply function recursively
            continue

        #where are the initializers?
        if hasattr(layer, 'cell'):
            init_container = layer.cell
        else:
            init_container = layer

        for key, initializer in init_container.__dict__.items():
            if "initializer" not in key: #is this item an initializer?
                  continue #if no, skip it

            # find the corresponding variable, like the kernel or the bias
            if key == 'recurrent_initializer': #special case check
                var = getattr(init_container, 'recurrent_kernel')
            else:
                var = getattr(init_container, key.replace("_initializer", ""))

            var.assign(initializer(var.shape, var.dtype))
            #use the initializer

In [19]:
def train():
    clear_output()
    wandb.init()
    config = wandb.config

    default_config = wandb.config
    default_config.equation='tranp_equation'
    default_config.WENO='WENO-Z-DS'

    default_config.poly_grade=7
    default_config.seno_ampli=3 
    default_config.gauss_var=3
    default_config.seizures=3

    config = default_config

    equation_name='transp_equation'
    poly_grade=config.poly_grade
    seno_ampli=config.seno_ampli
    gauss_var=config.gauss_var
    seizures=config.seizures

    data_temporal,data_spatial,data_base,Δx,Δt,CFL,fronteira,equation=load_dataset(f'datasets/equation={equation_name} polys grau={poly_grade} seno_ampli={seno_ampli} gauss_var={gauss_var} seizure={seizures}')

    indice = np.arange(data_temporal.shape[0])
    np.random.shuffle(indice)
    data_x = data_temporal.astype('float64')[indice]
    data_y = data_temporal.astype('float64')[indice]

    train_x = data_x[:,0:-1]
    train_y = data_y[:,1:]

     # get_initializer() returns a callable, seed can be set to None for random weights
    reset_weights(Network)

    for epoch in range(1):
        loss=Network.fit(
                train_x                           , # Dados de treino
                train_y                           , # Dados de treino
                batch_size      = 4               , # Tamanho do batch
                epochs          = 1              , # Número de epochs
                steps_per_epoch = 10              , # Número de epochs
                shuffle         = True            ,  # Aleatorização dos batchs
                callbacks=[WandbCallback()]
            )
        #wandb.log({"loss": loss, "epoch": epoch})
    Δt=0.01*CFL
    total_time=int(2*1/Δt)
    prev_fx=fx[...]
    count=0
    for i in range(total_time):
        if(i%10==0):
            print(count,end='\r')
        prev_fx=Sim_layer.Sim_step(fx,Δt, 0.01, FronteiraPeriodica)
        count+=Δt

    #prev_fx=Sim_layer.Sim(fx,2*(3), 0.01, CFL, FronteiraPeriodica)
    erro=2*tf.math.reduce_mean(tf.abs(prev_fx-fx))/0.01
    
    wandb.log({'validation_error':erro})

#wandb.agent(sweep_id, train)

In [29]:
wandb.agent(sweep_id, train)

{'gauss_var': 0, 'poly_grade': 3, 'seizures': 2, 'seno_ampli': 1}
'Δx': 0.015625
'Δt': 0.004
'CFL': 0.5
'fronteira': <function FronteiraPeriodica at 0x0000015D86B7F550>
'equation': <class 'aux_equation.transp_equation'>
'seed': None

Epoch 1/30
 5/38 [==>...........................] - ETA: 15:16 - loss: 1.6694e-04 - mean_absolute_error: 0.0014

wandb: Ctrl + C detected. Stopping sweep.


In [20]:
train()

epoch,▁
loss,▁
mean_absolute_error,▁
epoch,0
loss,0.00013
mean_absolute_error,0.00149


'Δx': 0.015625
'Δt': 0.004
'CFL': 0.5
'fronteira': <function FronteiraPeriodica at 0x0000015D86B7F550>
'equation': <class 'aux_equation.transp_equation'>
'seed': None

10/10 [==============================] - 13s 1s/step - loss: 1.3470e-04 - mean_absolute_error: 0.0016 - _timestamp: 1651018212.0000 - _runtime: 30.0000
